In [ ]:
# Celda 1: inspección de estructura
import os, glob
import numpy as np

ROOT = "../data/Tinamidae_5s_npz"

species_dirs = sorted([d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))])
counts = {sp: len(glob.glob(os.path.join(ROOT, sp, "*.npz"))) for sp in species_dirs}

print(f"Especies encontradas: {len(species_dirs)}")
print("Ejemplos de conteos (5 spp):", list(counts.items())[:5])
print("Rango de archivos por especie:",
      min(counts.values()) if counts else None, "—",
      int(np.median(list(counts.values()))) if counts else None, "—",
      max(counts.values()) if counts else None)


In [ ]:
# Celda 2: función para cargar un vector desde un .npz 
import numpy as np, os, glob

def load_vec_from_npz(path):
    with np.load(path, allow_pickle=False) as data:
        if 'embedding' in data:
            arr = data['embedding']
        elif 'arr_0' in data:
            arr = data['arr_0']
        else:
            key = list(data.keys())[0]
            arr = data[key]
    arr = np.asarray(arr).squeeze()
    if arr.ndim > 1:
        arr = arr.reshape(-1)
    return arr

# Prueba con un archivo cualquiera
if species_dirs:
    test_sp = species_dirs[0]
    test_file = glob.glob(os.path.join(ROOT, test_sp, "*.npz"))[0]
    v = load_vec_from_npz(test_file)
    print("Prueba:", test_sp, os.path.basename(test_file), "shape:", v.shape)
    assert v.ndim == 1, "El embedding debería ser un vector 1D"


In [ ]:
# Celda 3: construir el CSV 46x1280 usando la MEDIANA por especie
import os, glob, numpy as np, pandas as pd

OUT_DIR = "../traits"
OUT_CSV = os.path.join(OUT_DIR, "traits_high-level.csv")
os.makedirs(OUT_DIR, exist_ok=True)

summary = {}
dims_set = set()

for sp in species_dirs:
    files = sorted(glob.glob(os.path.join(ROOT, sp, "*.npz")))
    vecs = []
    for f in files:
        v = load_vec_from_npz(f)
        v = np.asarray(v).reshape(-1)      # aplastar por si viene (1,1280) o (1280,1)
        vecs.append(v)
        dims_set.add(v.shape[0])
    if not vecs:
        continue
    M = np.stack(vecs, axis=0)             # (n_samples, d)
    med = np.median(M, axis=0)             # resumen robusto
    summary[sp] = med

df = pd.DataFrame.from_dict(summary, orient="index")
df.index.name = "species"

# Forzar nombres de columnas e1..e1280 si la dimensión es 1280
if df.shape[1] == 1280:
    df.columns = [f"e{i:04d}" for i in range(1, 1281)]

df.to_csv(OUT_CSV, float_format="%.6f")
print("Guardado:", OUT_CSV)
print("Shape del CSV:", df.shape)
print("Dimensiones encontradas:", dims_set)
print(df.iloc[:3, :5])  


In [ ]:
# Celda 4: Selección de 10 muestras por especie (gradiente radial) + 10 CSVs (uno por set)
import os, glob
import numpy as np
import pandas as pd

OUT_DIR   = "../traits"
SETS_DIR  = os.path.join(OUT_DIR, "gradient_sets")
SEL_CSV   = os.path.join(OUT_DIR, "selected_samples_gradient.csv")
os.makedirs(SETS_DIR, exist_ok=True)

N_SETS = 10
Q = np.linspace(0.05, 0.95, N_SETS)  # cuantiles (trim) para evitar outliers extremos
METRIC = "cosine"  # "cosine" o "euclidean"

def l2_normalize(X, axis=1, eps=1e-12):
    n = np.linalg.norm(X, axis=axis, keepdims=True)
    return X / (n + eps)

def distances_to_median(M, med, metric="cosine"):
    if metric == "euclidean":
        return np.linalg.norm(M - med, axis=1)
    elif metric == "cosine":
        M2 = l2_normalize(M, axis=1)
        med2 = med / (np.linalg.norm(med) + 1e-12)
        return 1.0 - (M2 @ med2)  # 1 - cos(sim)
    else:
        raise ValueError("metric debe ser 'euclidean' o 'cosine'")

# Aquí acumulamos: (a) el “mapa” de qué archivo fue elegido, y (b) los 10 datasets finales
meta_rows = []
set_vectors = {k: {} for k in range(N_SETS)}  # set_vectors[k][species] = vector

# Diagnóstico: ¿alcanzan los archivos por especie para 10 únicos?
counts = {sp: len(glob.glob(os.path.join(ROOT, sp, "*.npz"))) for sp in species_dirs}
min_n = min(counts.values()) if counts else 0
if min_n < N_SETS:
    print(f"WARNING: hay especies con menos de {N_SETS} archivos (mínimo={min_n}).")
    print("         En esas especies habrá selección con reemplazo (repetidos) para completar sets.")

for sp in species_dirs:
    files = sorted(glob.glob(os.path.join(ROOT, sp, "*.npz")))
    if not files:
        continue

    vecs = [np.asarray(load_vec_from_npz(f)).reshape(-1) for f in files]
    M = np.stack(vecs, axis=0)      # (n, d)
    med = np.median(M, axis=0)      #  “centro”

    dist = distances_to_median(M, med, metric=METRIC)
    targets = np.quantile(dist, Q)

    chosen = set()
    chosen_idx = []

    for k, t in enumerate(targets):
        # ordena candidatos por cercanía al target
        cand = np.argsort(np.abs(dist - t))
        pick = None
        for idx in cand:
            if idx not in chosen:
                pick = int(idx)
                break
        if pick is None:
            # no quedan únicos (n < N_SETS): usar reemplazo
            pick = int(cand[0])

        chosen.add(pick)
        chosen_idx.append(pick)

        meta_rows.append({
            "species": sp,
            "set": k + 1,
            "quantile": float(Q[k]),
            "distance": float(dist[pick]),
            "file": files[pick]
        })
        set_vectors[k][sp] = vecs[pick]

# Guardar tabla “qué se eligió”
pd.DataFrame(meta_rows).to_csv(SEL_CSV, index=False)
print("Guardado:", SEL_CSV)

# Guardar 10 CSVs (uno por set)
for k in range(N_SETS):
    dfk = pd.DataFrame.from_dict(set_vectors[k], orient="index")
    dfk.index.name = "species"
    if dfk.shape[1] == 1280:
        dfk.columns = [f"e{i:04d}" for i in range(1, 1281)]
    out_csv = os.path.join(SETS_DIR, f"traits_high-level_set{k+1:02d}.csv")
    dfk.to_csv(out_csv, float_format="%.6f")
    print("Guardado:", out_csv, "| shape:", dfk.shape)